# Adding support for new devices

Currently there are two supported devices: **Arduino Nano 33 BLE Sense Lite** and **Arducam Pico4ML**.

Supported means that the webapp is capable of building and uploading a person detection application on to the device, and predictions can be read from the device. Obviously the process will change as new features are added and more is required for adding support. 

## Steps to adding support for a new device

### 1. Find example code and tutorials for the device

Figure out how to compile and upload code to the device. Get some basic hello world or LED blink example to run on the device. Ideally you should be able to do this from the command line, since that is eventually required for the Dockerfile.

### 2. Compile the [tflite-micro person_detection example](https://github.com/tensorflow/tflite-micro/tree/main/tensorflow/lite/micro/examples/person_detection) for the device

Generally it seems to be easier to use a device-specific tflite-micro repository instead of the main repository.
We used [tflite-micro-arduino-examples](https://github.com/tensorflow/tflite-micro-arduino-examples) and [RPI-Pico-Cam](https://github.com/ArduCAM/RPI-Pico-Cam) with the current devices.

You might need to add device-specific code for taking pictures with the camera and printing values to serial. Device-specific repositories often already include this code.

### 3. Modify the example to work with models created by TinyMLaaS

You need to make the following changes to the example code:

- **(main_functions.cpp)** `kTensorArenaSize` should be set to at least 182 kilobytes (this is the amount of memory that must be given to tflite-micro for our model to work).

- **(main_functions.cpp)** The micro_op_resolver variable should be modified to contain all 11 operations needed by our model. You can copy its definition from arduino/template/template.ino (These will have to be changed if the model training code is modified).

- **(person_detect_model_data.h)** The extern declarations should match what is generated by nbs/compiling.ipynb. You can copy the contents of arduino/template/person_detect_model_data.h.

- **(main_functions.cpp)** The model passed to `tflite::GetModel` should be the array from the previous step. `model_tflite` is the currently used name for it.

- **(person_detect_model_data.cpp)** replace this file with a **model.cc** created by the webapp. You can obtain one by copying it out of the frontend docker container after running training and compiling in the webapp. You could also just copy rpi-pico/code/person_detection_screen/person_detection_data.cpp which was created this way. 

- **(detection_responder.h / detection_responder.cpp / main_functions.cpp)** Modify the `RespondToDetection` function (and the code calling it) to print detections in the format expected by nbs/observing.ipynb. You can copy the code from one of the existing devices.

- Our model currently operates on grayscale images, make sure that that's what the camera code is producing. We've also only used 96x96 images so you might need to change something else if the camera resolution is different.

In theory you should now have the person_detection app running with a custom model, but in our experience this is the part where the app mysteriously stops working. Here are the issues that we encountered:

- **kTensorArenaSize is too small**: this causes an error message to get printed via serial on setup(), and causes the inference code to repeatedly print `Invoke failed` errors to serial. 

- **micro_op_resolver doesn't have the necessary ops**: this causes an error message to get printed via serial on setup(), and causes the inference code to repeatedly print `Invoke failed` errors to serial. This should not happen if you followed the steps above.

- **Unaligned model data**: The app was crashing inside tensorflow's AllocateTensors function with no errors. The fix was to add `alignas(16)` specifier to the model data array. This is currently not done on either device, so this might only be needed with older tflite-micro versions.

- **Outdated tflite-micro**: The app was crashing inside tensorflow's AllocateTensors function with no errors. We were initially using a library called "Harward TinyMLx" with Arduino. This contained both tflite-micro and camera code for the device, however the tflite version was outdated. Switching to a newer tflite-micro version fixed the issue. A quick way to tell the version the example is using is to look at how printing is done. If you see mentions of `tflite::ErrorReporter`/`TF_LITE_REPORT_ERROR`, the code is using the old version. Newer versions use `MicroPrintf` instead.

- **Running out of memory**: The Pico4ML was crashing inside the GetImage function with no error messages. This happened because the camera code was internally creating a 324x324 array on the stack, which combined with our increased kTensorArenaSize was causing the device to not have enough memory (presumably the stack was overflowing). Reducing the size of the array in the camera code fixed this issue.

### 4. Create a dockerfile for building the code

The dockerfile should produce an image that contains the compiled person_detection application, which can then be installed by the relay with only docker.

### 5. Modify the Webapp to build the code

- Create a subclass of InstallerImageBuilder in nbs/installing.ipynb to pass the model file to the dockerfile, build the docker image, and upload it to dockerhub.

- Edit the devices dictionary in pages/6_Installing.py to include your new device, its InstallerImageBuilder class, and a relay_id that will be used to refer to the device on the relay server side.

- Edit the install_inference function in relay/components/install.py to install the model on to the device using the docker image downloaded from dockerhub.

# Debugging Tip

Printing messages to serial is very useful for debugging. You can use them to determine values of variables or where the program is crashing. Also the tflite-micro library internally uses its MicroPrintf function to print error messages to serial, so reading from serial is also useful for finding out the cause of problems with tflite.

You can read data from serial with a tool called minicom: `minicom -b 115200 -D /dev/ttyACM0`